In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import notMNIST_gen

In [2]:
save = notMNIST_gen.gen_composite()
train_dataset = save['train_dataset']
train_labels = save['train_labels']
valid_dataset = save['valid_dataset']
valid_labels = save['valid_labels']
test_dataset = save['test_dataset']
test_labels = save['test_labels']
del save  # hint to help gc free up memory
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Starting
Download Complete
Extract Complete
Saving Complete
Data Dictionaries Built
Training set (200000, 64, 64) (200000,)
Validation set (10000, 64, 64) (10000,)
Test set (10000, 64, 64) (10000,)


In [3]:
print(train_dataset.dtype)

uint8


In [4]:
image_size = 64
num_digits = 5
num_labels = 11
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  l_shape = labels.shape
  new_labels = np.ndarray((l_shape[0], num_digits))
  for l, label in enumerate(labels):
    for c, char in enumerate(label):
        new_labels[l,c] = char
  labels = (np.arange(num_labels) == new_labels[:,:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 64, 64, 1) (200000, 5, 11)
Validation set (10000, 64, 64, 1) (10000, 5, 11)
Test set (10000, 64, 64, 1) (10000, 5, 11)


In [5]:
print(train_dataset.dtype)

float32


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.all(np.argmax(predictions, 2) == np.argmax(labels, 2), 1))
          / predictions.shape[0])

In [2]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels), 
      name='train_dataset_in')
  tf_train_labels = tf.placeholder(
      tf.float32, shape=(batch_size, num_digits, num_labels), name='train_labels_in')
  tf_valid_dataset = tf.constant(valid_dataset, name='valid_dataset_in')
  tf_test_dataset = tf.constant(test_dataset, name='valid_labels_in')
  
  # Variables.
  def variable_dict(digit):
      var_dict = {}
      var_dict['layer1_weights'] = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1), 
                                               name='layer1_'+ digit + '_weights')
      var_dict['layer1_biases'] = tf.Variable(tf.zeros([depth]), 
                                              name='layer1_'+ digit + '_biases')
      var_dict['layer2_weights'] = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1), 
                                               name='layer2_'+ digit + '_weights')
      var_dict['layer2_biases'] = tf.Variable(tf.constant(1.0, shape=[depth]), 
                                              name='layer1_'+ digit + '_biases')
      var_dict['layer3_weights'] = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1), 
                                               name='layer3_'+ digit + '_weights')
      var_dict['layer3_biases'] = tf.Variable(tf.constant(1.0, shape=[num_hidden]), 
                                              name='layer3_'+ digit + '_biases')
      var_dict['layer4_weights'] = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1), name='layer4_'+ digit + '_weights')
      var_dict['layer4_biases'] = tf.Variable(tf.constant(1.0, shape=[num_labels]), 
                                              name='layer1_'+ digit + '_biases')
      return var_dict

  digit_1_vars = variable_dict('1')
  digit_2_vars = variable_dict('2')
  digit_3_vars = variable_dict('3')
  digit_4_vars = variable_dict('4')
  digit_5_vars = variable_dict('5')
                      
  # Model.
  def char_model(data, var_set):
    conv = tf.nn.conv2d(data, var_set['layer1_weights'], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.max_pool(conv, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + var_set['layer1_biases'])
    conv = tf.nn.conv2d(hidden, var_set['layer2_weights'], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.max_pool(conv, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + var_set['layer2_biases'])
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, var_set['layer3_weights']) + var_set['layer3_biases'])
    result = tf.matmul(hidden, var_set['layer4_weights']) + var_set['layer4_biases']
    return result
  
  def model(data):
      logit_1 = char_model(data, digit_1_vars)
      logit_2 = char_model(data, digit_2_vars)
      logit_3 = char_model(data, digit_3_vars)
      logit_4 = char_model(data, digit_4_vars)
      logit_5 = char_model(data, digit_5_vars)    
      logit_stack = [logit_1, logit_2, logit_3, logit_4, logit_5]
      return (tf.pack(logit_stack, axis=1))

  # Training computation.
  logits = model(tf_train_dataset)
  logit_shape = logits.get_shape().as_list()
  label_shape = tf_train_labels.get_shape().as_list()
  logit_reshape = tf.reshape(logits,[logit_shape[0]*logit_shape[1], logit_shape[2]])
  label_reshape = tf.reshape(tf_train_labels,[label_shape[0]*label_shape[1], label_shape[2]])
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logit_reshape, label_reshape))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.0001, global_step, 500, 0.99)
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

NameError: name 'tf' is not defined

In [ ]:
num_steps = 11

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :, :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 0.000000
Minibatch accuracy: 0.0%
